# Simple Q&A with Qwen 7B

Notebook này hướng dẫn load và sử dụng Qwen 7B model để thực hiện Q&A đơn giản.

## Thông tin
- **Model**: Qwen/Qwen2.5-7B-Instruct
- **Task**: Question Answering


In [1]:
# Install dependencies (chạy cell này trước nếu chưa cài đặt)

import sys
import subprocess

def check_and_install(package_name):
    """Kiểm tra và cài đặt package nếu chưa có"""
    try:
        __import__(package_name.split('>=')[0].split('==')[0].split('<')[0].strip())
        print(f"✓ {package_name} đã được cài đặt")
        return True
    except ImportError:
        print(f"⚠ {package_name} chưa được cài đặt. Đang cài đặt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"✓ {package_name} đã được cài đặt")
        return False

print("Đang kiểm tra và cài đặt dependencies...\n")

packages = [
    "torch",
    "transformers>=4.35.0",
    "accelerate>=0.24.0",
]

for package in packages:
    check_and_install(package)

print("\n" + "="*50)
print("✓ Tất cả dependencies đã sẵn sàng!")
print("="*50)


Đang kiểm tra và cài đặt dependencies...

✓ torch đã được cài đặt
✓ transformers>=4.35.0 đã được cài đặt
✓ accelerate>=0.24.0 đã được cài đặt

✓ Tất cả dependencies đã sẵn sàng!


## Bước 1: Import thư viện


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Kiểm tra GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Sử dụng CPU")


CUDA available: True
CUDA device: Tesla T4
CUDA memory: 15.83 GB


## Bước 2: Load Model và Tokenizer


In [3]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
    trust_remote_code=True
)

# Move to device if not using device_map
if not torch.cuda.is_available():
    model = model.to("cpu")

print(f"✓ Model loaded: {model_name}")
print(f"✓ Model device: {next(model.parameters()).device}")


Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✓ Model loaded: Qwen/Qwen2.5-7B-Instruct
✓ Model device: cuda:0


## Bước 3: Hàm Q&A


In [ ]:
def ask_question(question: str, max_new_tokens: int = 512, temperature: float = 0.7):
    """
    Hàm để hỏi đáp với model
    
    Args:
        question: Câu hỏi cần trả lời
        max_new_tokens: Số token tối đa để generate
        temperature: Nhiệt độ cho sampling (0.0-1.0)
    """
    # Format prompt cho Qwen Instruct model
    messages = [
        {"role": "user", "content": question}
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True if temperature > 0 else False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode response
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

print("✓ Hàm ask_question đã sẵn sàng!")


✓ Hàm ask_question đã sẵn sàng!


## Bước 4: Test Q&A


In [5]:
# Test với một số câu hỏi đơn giản
test_questions = [
    "Xin chào! Bạn là ai?",
    "Python là gì?",
    "Giải thích ngắn gọn về machine learning",
]

print("="*60)
print("TESTING Q&A")
print("="*60)

for i, question in enumerate(test_questions, 1):
    print(f"\n[{i}] Câu hỏi: {question}")
    print("-" * 60)
    answer = ask_question(question)
    print(f"Trả lời: {answer}")
    print("="*60)


TESTING Q&A

[1] Câu hỏi: Xin chào! Bạn là ai?
------------------------------------------------------------
Trả lời: Xin chào! Tôi là Qwen, một mô hình ngôn ngữ lớn được tạo ra bởi Alibaba Cloud. Tôi có thể giúp bạn với nhiều câu hỏi và nhiệm vụ liên quan đến ngôn ngữ tự nhiên. Bạn có câu hỏi hay chủ đề nào muốn thảo luận không?

[2] Câu hỏi: Python là gì?
------------------------------------------------------------
Trả lời: Python là một ngôn ngữ lập trình phổ biến và linh hoạt được thiết kế để dễ đọc và viết. Dưới đây là một số đặc điểm chính của Python:

1. **Dễ học và sử dụng**: Python có cú pháp đơn giản, dễ hiểu, phù hợp cho cả người mới bắt đầu và các nhà phát triển chuyên nghiệp.

2. **Đa mục đích**: Có thể dùng để phát triển web, phân tích dữ liệu, trí tuệ nhân tạo, khoa học máy tính, tự động hóa, và nhiều lĩnh vực khác.

3. **Thư viện đa dạng**: Python có một thư viện mở rộng lớn, giúp giải quyết hầu hết mọi nhu cầu lập trình.

4. **Hỗ trợ lập trình hướng đối tượng**: Cho phé

## Sử dụng tùy chỉnh

Bạn có thể sử dụng hàm `ask_question()` với bất kỳ câu hỏi nào:


In [ ]:
# Thử với câu hỏi của bạn
my_question = "Hãy giải thích về transformer architecture trong deep learning"

answer = ask_question(my_question, max_new_tokens=256, temperature=0.7)
print(f"Câu hỏi: {my_question}")
print(f"\nTrả lời:\n{answer}")
